<h1>Cek Kata Kotor</h1>
<p>Albert Lilian Thamson / 2201754412</p>
<p>Daniel Santoso / 2201756506</p>
<p>Luwis Lim / 2201761771</p>
<p>Steven Odolf Yuwono / 2201758045</p>

In [1]:
# Import Stopwords
f = open('stopwordsindo.txt', 'r')
content = f.read()
content = list(content.split("\n"))

print(content)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Masukkan data
df = pd.read_csv('clean_data.csv')
komentar = df['Comments'].astype(str)
y = df['Kotor']

# Vektorisasi teksnya
vectorizer = CountVectorizer(stop_words = frozenset(content), min_df = 0.0001)
X = vectorizer.fit_transform(komentar)

x1, x2, y1, y2 = train_test_split(X, y, random_state = 0, train_size = 0.2)

# Cek ukuran dari setiap komponen
print(x1.shape)
print(y1.shape)
print(x2.shape)
print(y2.shape)

(197, 2369)
(197,)
(792, 2369)
(792,)


C:\Users\Steve\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\Steve\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [3]:
# Train modelnya
model = LinearSVC(class_weight = "balanced", dual = False, tol = 1e-2, max_iter = 1e5)
cclf = CalibratedClassifierCV(base_estimator=model)
cclf.fit(x1, y1)

pred = cclf.predict(x2)
print("Akurasi :", accuracy_score(y2, pred))

Akurasi : 0.7171717171717171


In [4]:
# Simpan model
joblib.dump(vectorizer, 'data/vectorizer.joblib')
joblib.dump(cclf, 'data/model.joblib')

['data/model.joblib']

In [5]:
import pkg_resources
import joblib
import numpy as np

vectorizer = joblib.load(pkg_resources.resource_filename('cekkotor', 'data/vectorizer.joblib'))
model = joblib.load(pkg_resources.resource_filename('cekkotor', 'data/model.joblib'))

def _get_prob_kotor(prob):
  return prob[1]

def prediksi(kalimat):
  return model.predict(vectorizer.transform(kalimat))

def prediksi_prob(kalimat):
  return np.apply_along_axis(_get_prob_kotor, 1, model.predict_proba(vectorizer.transform(kalimat)))

def kotorAtauTidak(hasil_pred):
    if(hasil_pred==0):
        return 'BERSIH'
    else:
        return 'KOTOR'

C:\Users\Steve\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Steve\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Steve\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\Steve\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: 

In [6]:
## PERCOBAAN

print('---HASIL PERCOBAAN---')
print('')

print('"aku suka kamu" --> ' + kotorAtauTidak(prediksi(['aku suka kamu'])))
print('probabilitas: {}'.format(prediksi_prob(['aku suka kamu'])))
print('')
print('"kamu memang hebat, jangan ragu!" --> ' + kotorAtauTidak(prediksi(['kamu memang hebat, jangan ragu!'])))
print('probabilitas: {}'.format(prediksi_prob(['kamu memang hebat, jangan ragu!'])))
print('')
print('"bacot anjing jembut" --> ' + kotorAtauTidak(prediksi(['bacot anjing jembut'])))
print('probabilitas: {}'.format(prediksi_prob(['bacot anjing jembut'])))
print('')
print('"elu bajingan asu anjing cuiiihhh babi kontol lu kecil gamau gua ama elu" --> ' + kotorAtauTidak(prediksi(['eriko kayak bajingan asu anjing cuiiihhh babi kontol lu kecil'])))
print('probabilitas: {}'.format(prediksi_prob(['elu bajingan asu anjing cuiiihhh babi kontol lu kecil gamau gua ama elu'])))
print('')

---HASIL PERCOBAAN---

"aku suka kamu" --> BERSIH
probabilitas: [0.4745261]

"kamu memang hebat, jangan ragu!" --> BERSIH
probabilitas: [0.39176284]

"bacot anjing jembut" --> KOTOR
probabilitas: [0.77421065]

"elu bajingan asu anjing cuiiihhh babi kontol lu kecil gamau gua ama elu" --> KOTOR
probabilitas: [0.87148834]



C:\Users\Steve\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
